
# (Original) Data Overview

## Target Variable:
- belief: category, the religious belief status of the individual, used to classify Atheism(0) and Theism(1).

## Continuous/Numeric Variables:
- age: int64, the age of the user.
- height: float64, the height of the user in centimeters.
- income: int64, the income of the user.
- seriousness_degree: float64, the degree of seriousness about religion. (0 = unspecified ,1-4 = not serious to very serious)

## Categorical Variables:
- status: category, relationship status.
- sex: category, gender.
- orientation: category, sexual orientation.
- body_type: category, body type.
- diet: category, dietary habits.
- drinks: category, alcohol consumption habits.
- drugs: category, drug usage.
- education: category, educational attainment.
- ethnicity: category, ethnic background.
- job: category, employment description.
- location: category, user location.
- offspring: category, children status.
- pets: category, pet preferences.
- religion: category, religious background.
- sign: category, astrological symbol.
- smokes: category, smoking consumption.
- speaks: category, language spoken.
- education_clean: category, cleaned and simplified education information.
- education_final: category, finalized education information after processing.
- cleaned_religion: category, cleaned version of religious background.

## Text Variables:
- My self summary: object, a summary written by the user about themselves.
- What I’m doing with my life: object, user's description of their current life status.
- I’m really good at: object, what the user describes as their strengths.
- The first thing people usually notice about me: object, what the user thinks people notice about them first.
- Favorite books, movies, show, music, and food: object, user's favorites in books, movies, shows, music, and food.
- The six things I could never do without: object, things the user considers essential in their life.
- I spend a lot of time thinking about: object, topics the user often thinks about.
- On a typical Friday night I am: object, user's typical activities on a Friday night.
- The most private thing I am willing to admit: object, something personal the user is willing to share.
- You should message me if…: object, user's criteria for others to contact them.
- merged_profile: object, combined text from all essay sections.


# Feature Selection

## In this task, the goal is to predict one's religious belief based on the following features:

#8 Catgorical:
diet(clean), 
drinks,
etnicity(clean), 
education(final), 
status, 
drugs, 
smokes, 
orientation,
job(clean)

#2 Num:
age, 
height

$ Tagert:
belief

## Since the data transforming process is completed in previous taks, I will extract the OH-Encoded Feature Matrix (with target variable directly)

In [58]:

import numpy as np
import pandas as pd
from collections import Counter
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

%matplotlib inline

In [59]:
# read csv file

df = pd.read_csv('/Users/adhdtreamentii/Desktop/UChicago 2024/UChicago Winter 24/MACSS-30100/Final/data/encoded_cleanedcat_dataX.csv')

### Basic dataset exploration

In [60]:
# display the dataset
print(df.shape)
df.head()

(16597, 74)


,Unnamed: 0,age,height,belief,diet_clean_anything,diet_clean_halal,diet_clean_kosher,diet_clean_other,diet_clean_unknown,diet_clean_vegan,...,job_clean_military,job_clean_other,job_clean_political,job_clean_rather not say,job_clean_retired,job_clean_sales,job_clean_science,job_clean_student,job_clean_transportation,job_clean_unemployed
0,7,31,65.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,9,37,65.0,0,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
2,11,28,72.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,13,30,66.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0
4,14,29,62.0,1,1.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [61]:
# drop the original index column,  the 'Unnamed: 0' column
df = df.drop(['Unnamed: 0'], axis=1)

In [62]:
print(df.shape)
df.columns

(16597, 73)


Index(['age', 'height', 'belief', 'diet_clean_anything', 'diet_clean_halal',
       'diet_clean_kosher', 'diet_clean_other', 'diet_clean_unknown',
       'diet_clean_vegan', 'diet_clean_vegetarian', 'drinks_desperately',
       'drinks_not at all', 'drinks_often', 'drinks_rarely', 'drinks_socially',
       'drinks_very often', 'ethnicity_clean_asian', 'ethnicity_clean_black',
       'ethnicity_clean_indian', 'ethnicity_clean_latin',
       'ethnicity_clean_middle eastern', 'ethnicity_clean_multiethnic',
       'ethnicity_clean_native american', 'ethnicity_clean_other',
       'ethnicity_clean_pacific islander', 'ethnicity_clean_unknown',
       'ethnicity_clean_white', 'education_final_college/university',
       'education_final_high school', 'education_final_law school',
       'education_final_masters program', 'education_final_med school',
       'education_final_ph.d program', 'education_final_space camp',
       'education_final_two-year college', 'education_final_unknown',
     

In [63]:
# First we need to identify the target variable and the feature set.

# Check if the 'belief' column is in df_X
if 'belief' in df.columns:
    df_X = df.drop('belief', axis=1)

# Update feature names
feature_names = df_X.columns.to_list()

print(df_X.shape)


(16597, 72)


In [64]:
# Create target variable
target = 'belief'
df_y = df[target]
df_y.shape

(16597,)

In [65]:
# Create split with Sklearn
df_X_train, df_X_test, df_y_train, df_y_test  = train_test_split(df_X, df_y, test_size = 0.3,random_state = 42)

# View the training and testing feature matrix 
df_X_train.shape, df_X_test.shape

((11617, 72), (4980, 72))

## Linear models for classification: LogisticRegression 


### For this binary classification task, Logistic Regression is generally preferred over linear models and polynomial regression due to several key reasons. 
### Firstly, Logistic Regression is specifically designed for binary classification, as it models the probability of a binary outcome using a logistic function, ensuring that the predicted values are sensibly confined between 0 and 1. This is crucial for binary classification tasks where the output is categorical (e.g.,theism/atheism). In contrast, linear models, which predict continuous values, are NOT naturally suited for predicting probabilities and require additional steps to threshold the output for classification. Polynomial regression, while capable of modeling more complex relationships than linear models, still suffers from the same limitation in a binary classification context. 
### Logistic Regression also offers a straightforward interpretation of the relationship between features and the probability of outcomes, which is less intuitive in polynomial models due to their complexity. Furthermore, Logistic Regression can be easily extended to handle multi-class classification problems, making it a better choice for classification tasks.

### Fit LogisticRegression models with different parameter settings
- L1 VS L2 penalty
- C values (inverse of regularization strength)

In [71]:
clf = LogisticRegression(random_state=0, solver='liblinear', penalty='l1', C=1.0).fit(df_X_train, df_y_train)

In [73]:
# training accuracy
clf.score(df_X_train, df_y_train)

0.7636222776964793

In [75]:
# testing accuracy
clf.score(df_X_test, df_y_test)

0.7602409638554217

### training accuracy and testing accuracy indicate that the model performance is pretty consistent, so the model is not overfitting

In [83]:
np.round(clf.predict_proba(df_X_test[:3]),3)

array([[0.117, 0.883],
       [0.141, 0.859],
       [0.202, 0.798]])

In [77]:
# confusion matrix
from sklearn.metrics import confusion_matrix
y_pred = clf.predict(df_X_test)
confusion_matrix(df_y_test, y_pred)

# classification report
from sklearn.metrics import classification_report
print(classification_report(df_y_test, y_pred))




              precision    recall  f1-score   support

           0       0.66      0.32      0.43      1416
           1       0.78      0.93      0.85      3564

    accuracy                           0.76      4980
   macro avg       0.72      0.63      0.64      4980
weighted avg       0.74      0.76      0.73      4980



In [78]:
df['belief'].value_counts()

1    12029
0     4568
Name: belief, dtype: int64

### The recall for classifying the 0-class (the minority class) is unusually low. This is a common issue in imbalanced datasets. The model, having been exposed to more instances of class 1, becomes better at identifying class 1 and less effective at recognizing class 0. As a result, even if the overall accuracy of the model seems high, its ability to correctly identify instances of the minority class (class 0) is compromised, leading to a low recall for that class.

In [91]:
# features were used in training
clf.feature_names_in_

array(['age', 'height', 'diet_clean_anything', 'diet_clean_halal',
       'diet_clean_kosher', 'diet_clean_other', 'diet_clean_unknown',
       'diet_clean_vegan', 'diet_clean_vegetarian', 'drinks_desperately',
       'drinks_not at all', 'drinks_often', 'drinks_rarely',
       'drinks_socially', 'drinks_very often', 'ethnicity_clean_asian',
       'ethnicity_clean_black', 'ethnicity_clean_indian',
       'ethnicity_clean_latin', 'ethnicity_clean_middle eastern',
       'ethnicity_clean_multiethnic', 'ethnicity_clean_native american',
       'ethnicity_clean_other', 'ethnicity_clean_pacific islander',
       'ethnicity_clean_unknown', 'ethnicity_clean_white',
       'education_final_college/university',
       'education_final_high school', 'education_final_law school',
       'education_final_masters program', 'education_final_med school',
       'education_final_ph.d program', 'education_final_space camp',
       'education_final_two-year college', 'education_final_unknown',
       '

## the top-10 features with highest coefficients in each model 

In [92]:
# ridge regression coefficients
df_linear_coef = pd.DataFrame({'features':clf.feature_names_in_, 'coefficients':np.round(clf.coef_[0],3)})

display(df_linear_coef.sort_values(['coefficients'], ascending=False).iloc[:10])

,features,coefficients
23,ethnicity_clean_pacific islander,1.766
4,diet_clean_kosher,1.731
3,diet_clean_halal,1.414
16,ethnicity_clean_black,1.390
40,drugs_never,0.899
62,job_clean_military,0.820
67,job_clean_sales,0.705
52,job_clean_banking,0.649
38,status_single,0.587
27,education_final_high school,0.568


## Compare model performance with different c values and different penalties

In [88]:
import numpy as np

def compare_c(X_train, y_train, X_test, y_test, p):
    """
    X_train/test: 2D feature matrix of training/testing data
    y_train/test: 1D array of training/testing labels
    p: the penalty parameter setting in LogisticRegression
    
    return: 
        a list of classifiers fitted with different c values
        a dataframe that is shown in the running example below
    """
     
    # set the model parameter c to different values and train the model 
    # for c in [0.001, 0.01, 0.1, 1, 10, 100]:
    #    fit a LogisticRegression model with: the current c value, the given penalty p, set random_state=42, max_iter=1000, solver='liblinear', and use default setting for other parameters
    #    test and record the model performance 
    #    get the statistical information about the model coefficients: 
    #        min: minimum coefficient
    #        max: minimum coefficient
    #        mean(abs(coef)): average over the absolute coefficient values
    #        n_zero: number of coefficients equal to zero 
    
    ### Your code starts from here 
    classifiers = []
    metrics_df = pd.DataFrame()
    

    # L2 penalty shows a very similar performance to L1 penalty
    for c in [0.001, 0.01, 0.1, 1, 10, 100]:
        clf = LogisticRegression(random_state=42, max_iter=1000, solver='liblinear', penalty=p, C=c)
        clf.fit(X_train, y_train)
        score = clf.score(X_test, y_test)
        coef_stats = {
            'min': np.min(clf.coef_),
            'max': np.max(clf.coef_),
            'mean_abs_coef': np.mean(np.abs(clf.coef_)),
            'n_zero': np.sum(clf.coef_ == 0)
        }
        classifiers.append(clf)
        metrics_df = metrics_df.append(coef_stats, ignore_index=True)

    return classifiers, metrics_df

In [95]:
# running example
l2_clfs, c_effect_l2 = compare_c(df_X_train, df_y_train, df_X_test, df_y_test, p='l2')
c_effect_l2

/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(coef_stats, ignore_index=True)
/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(coef_stats, ignore_index=True)
/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(coef_stats, ignore_index=True)
/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from

,min,max,mean_abs_coef,n_zero
0,-0.239483,0.248829,0.042189,1.0
1,-0.550316,0.540476,0.137296,1.0
2,-0.750964,1.038546,0.266685,1.0
3,-0.823471,1.567333,0.381639,1.0
4,-0.877220,1.772696,0.410909,1.0
5,-0.882531,1.794914,0.414134,1.0


In [96]:
# find the zero coefficients
feature_names = df_X_train.columns

for index, clf in enumerate(l2_clfs):
    coefs = clf.coef_[0]  # binary classification, so find the coefficients for the positive class
    zero_coefs_indices = np.where(coefs == 0)[0]
    zero_coef_features = feature_names[zero_coefs_indices]
    print(f"Model with C={clf.C}: Zero coefficients for features {list(zero_coef_features)}")


Model with C=0.001: Zero coefficients for features ['status_unknown']
Model with C=0.01: Zero coefficients for features ['status_unknown']
Model with C=0.1: Zero coefficients for features ['status_unknown']
Model with C=1: Zero coefficients for features ['status_unknown']
Model with C=10: Zero coefficients for features ['status_unknown']
Model with C=100: Zero coefficients for features ['status_unknown']


In [100]:

feature_names = df_X.columns

index_of_status_unknown = feature_names.get_loc('status_unknown')

prev_feature = feature_names[index_of_status_unknown - 1] if index_of_status_unknown > 0 else None
next_feature = feature_names[index_of_status_unknown + 1] if index_of_status_unknown < len(feature_names) - 1 else None

print(f"f-1: {prev_feature}")
print(f"f+1: {next_feature}")


f-1: status_single
f+1: drugs_never


this makes sense, because the 'unknown status' of someone's relationship status indeed provide no useful information with their religious belief

### As the value of C increases (which means a decrease in regularization strength), both the minimum and maximum values of the coefficients increase. This indicates that the model becomes more flexible, allowing the coefficients to take on larger values.

### The n_zero column shows the number of coefficients that are zero. In this example, in most cases, this value is 1, indicating that in most models, only one coefficient is zero. 

In [90]:
# running example
l2_clfs, c_effect_l2 = compare_c(df_X_train, df_y_train, df_X_test, df_y_test, p='l1')
c_effect_l2

/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(coef_stats, ignore_index=True)
/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(coef_stats, ignore_index=True)
/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  metrics_df = metrics_df.append(coef_stats, ignore_index=True)
/var/folders/sm/t07r9cb5347gwrxg17tf23th0000gn/T/ipykernel_45582/3898780944.py:41: FutureWarning: The frame.append method is deprecated and will be removed from

,min,max,mean_abs_coef,n_zero
0,-0.000883,0.032196,0.000459,70.0
1,-0.519564,0.867740,0.039202,61.0
2,-0.764510,1.088151,0.168119,32.0
3,-0.734953,1.768495,0.330467,9.0
4,-0.693469,2.478376,0.454191,3.0
5,-0.530438,2.682176,0.683228,1.0


### As the value of C increases, both the minimum and maximum values of the coefficients increase, consistent with the observations for L2 regularization. This indicates that as the regularization strength decreases, the model's coefficients are allowed to take on larger values.

### The most significant change is observed in the n_zero column, which represents the number of coefficients that are zero. For smaller values of C (stronger regularization), more coefficients become zero. This is a typical characteristic of L1 regularization, which tends to produce sparse models, aiding in feature selection.

In [103]:
feature_names = df_X_train.columns
top_coefs = []

for clf in l2_clfs:
    # get the coefficients
    coefs = clf.coef_[0]
    abs_coefs = np.abs(coefs)

    # find the top 3 coefficients in absolute values
    top_indices = np.argsort(-abs_coefs)[:3]
    top_coef_values = coefs[top_indices]
    top_features = feature_names[top_indices]

    # df for top coefficients
    for i in range(3):
        top_coefs.append({
            'C': clf.C,
            'Coefficient': top_coef_values[i],
            'Feature': top_features[i]
        })

top_coefs_df = pd.DataFrame(top_coefs)

top_coefs_df


,C,Coefficient,Feature
0,0.001,0.248829,drugs_never
1,0.001,-0.239483,ethnicity_clean_white
2,0.001,-0.205076,drugs_sometimes
3,0.010,-0.550316,job_clean_computer
4,0.010,-0.541969,ethnicity_clean_white
5,0.010,0.540476,drugs_never
6,0.100,1.038546,ethnicity_clean_black
7,0.100,0.853255,drugs_never
8,0.100,0.802451,ethnicity_clean_pacific islander
9,1.000,1.567333,ethnicity_clean_pacific islander


### The result of top coefs provides further evidences for my intial assumptions in EDA:
- the usages of drug is strongly associated with one's religous belief

### Other features (ethicity, diet habits)also play important roles in predicting belief, which are reasonable.